In [17]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

### Load dataset and preprocessing for Unet

In [36]:
!pip install --user kaggle
!pip install opendatasets

In [34]:
!mkdir -p ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c data-science-bowl-2018


UsageError: Line magic function `%%file` not found.


In [41]:
!kaggle competitions download -c data-science-bowl-2018


100%|█████████████████████████████████████████| 358M/358M [06:34<00:00, 945kB/s]
100%|█████████████████████████████████████████| 358M/358M [06:34<00:00, 952kB/s]


In [ ]:
#Download the dataset from kaggle
# Unzip the files and save locally


# Define the training and validation base directories
TRAIN_PATH = 'stage1_train/'
TEST_PATH = 'stage1_test/'

def load_image(datapoint):
    input_image = tf.image.resize(datapoint["images"], (IMG_WIDTH, IMG_HEIGHT))
    input_mask = tf.image.resize(datapoint["masks"], (IMG_WIDTH, IMG_HEIGHT),
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return input_image, input_mask

In [18]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3




In [23]:
# Build the model
inputs = tf.keras.layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS])

# Normalize the inputs
def normalizer(inputs):
    inputs = tf.cast(inputs, tf.float32) / 255.0
    return inputs

# Normalize alternative method
norm_input = tf.keras.layers.Lambda(lambda x: x/ 255.0)(inputs)

In [25]:
# contraction path
conv1 = tf.keras.layers.Conv2D(16, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(normalizer(norm_input))
conv1 = tf.keras.layers.Dropout(0.1)(conv1)
conv1 = tf.keras.layers.Conv2D(16, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(conv1)
pool1 = tf.keras.layers.MaxPool2D((2,2))(conv1)

conv2 = tf.keras.layers.Conv2D(32, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(pool1)
conv2 = tf.keras.layers.Dropout(0.1)(conv2)
conv2 = tf.keras.layers.Conv2D(32, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(conv2)
pool2 = tf.keras.layers.MaxPool2D((2,2))(conv2)

conv3 = tf.keras.layers.Conv2D(64, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(pool2)
conv3 = tf.keras.layers.Dropout(0.1)(conv3)
conv3 = tf.keras.layers.Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(conv3)
pool3 = tf.keras.layers.MaxPool2D((2,2))(conv3)

conv4 = tf.keras.layers.Conv2D(128, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(pool3)
conv4 = tf.keras.layers.Dropout(0.3)(conv4)
conv4 = tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(conv4)
pool4 = tf.keras.layers.MaxPool2D((2,2))(conv4)

conv5 = tf.keras.layers.Conv2D(256, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(pool4)
conv5 = tf.keras.layers.Dropout(0.3)(conv5)
conv5 = tf.keras.layers.Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(conv5)

# Expansive path
up6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(conv5)
up6 = tf.keras.layers.concatenate([up6, conv4])
conv6 = tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(up6)
conv6 = tf.keras.layers.Dropout(0.2)(conv6)
conv6 = tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(conv6)

up7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(conv6)
up7 = tf.keras.layers.concatenate([up7, conv3])
conv7 = tf.keras.layers.Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(up7)
conv7 = tf.keras.layers.Dropout(0.1)(conv7)
conv7 = tf.keras.layers.Conv2D(64, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(conv7)

up8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding="same")(conv7)
up8 = tf.keras.layers.concatenate([up8, conv2])
conv8 = tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(up8)
conv8 = tf.keras.layers.Dropout(0.1)(conv8)
conv8 = tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(conv8)

up9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides=(2,2), padding="same")(conv8)
up9 = tf.keras.layers.concatenate([up9, conv1], axis=3)
conv9 = tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(up9)
conv9 = tf.keras.layers.Dropout(0.1)(conv9)
conv9 = tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(conv9)



outputs = tf.keras.layers.Conv2D(1, (1,1), activation="sigmoid")(conv9)

model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

model.compile(optimizer="adam",
             loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda_4 (Lambda)              (None, 128, 128, 3)  0           ['input_7[0][0]']                
                                                                                                  
 tf.cast_9 (TFOpLambda)         (None, 128, 128, 3)  0           ['lambda_4[0][0]']               
                                                                                                  
 tf.math.truediv_9 (TFOpLambda)  (None, 128, 128, 3)  0          ['tf.cast_9[0][0]']        

                                                                                                  
 concatenate_10 (Concatenate)   (None, 64, 64, 64)   0           ['conv2d_transpose_10[0][0]',    
                                                                  'conv2d_115[0][0]']             
                                                                                                  
 conv2d_126 (Conv2D)            (None, 64, 64, 32)   18464       ['concatenate_10[0][0]']         
                                                                                                  
 dropout_57 (Dropout)           (None, 64, 64, 32)   0           ['conv2d_126[0][0]']             
                                                                                                  
 conv2d_127 (Conv2D)            (None, 64, 64, 32)   9248        ['dropout_57[0][0]']             
                                                                                                  
 conv2d_tr

In [ ]:
# define important callbacks
checkpointer = tf.keras.callbacks.ModelCheckpoint("model_for_nuclei.h5", verbose=1, save_best_only=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, verbose=1),
    tf.keras.callbacks.TensorBoard(log_dir="logs")
]
        
#Fit model
model.fit(X, Y, batch_size=16, epochs=25, validation_split=0.1 callbacks=callbacks)